In [47]:
#imports and stuff
import numpy as np
import sympy as sp

# Import the functions and packages that are used
from dwave.system import EmbeddingComposite, DWaveSampler
from dimod import BinaryQuadraticModel


In [50]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sp.Number)})

def generateQUBO(N,Nt,Nq,H):
    coefficients = H.coeffs()
    monomials = [sp.prod(x**k for x, k in zip(H.gens, mon)) for mon in H.monoms()]

    Q = np.zeros((N*Nt*Nq,N*Nt*Nq))
    diag_labels = np.array([])
    for n in range(N):
        for t in range(Nt):
            for q in range(Nq):
                mlabel = 'x_'+str(n)+'_'+str(t)+'_'+str(q)
                diag_labels = np.append(diag_labels,mlabel)
    
    print(diag_labels)
    for i in range(len(monomials)):
        mon = sp.Poly(monomials[i])
        gens = np.asarray(mon.gens)
        qubo_c = coefficients[i]
        
        qubo_i = np.where(diag_labels==str(gens[0]))[0][0]
        qubo_j = np.where(diag_labels==str(gens[0]))[0][0]
        
        if (len(gens) > 1):
            qubo_j = np.where(diag_labels==str(gens[1]))[0][0]
        
        Q[qubo_i,qubo_j] += qubo_c
                
    return Q

def generateWt(t,N,Nq,K):
    w_t = np.array([])
    for n in range(N):
        q_expr = 0
        for q in range(Nq):
            q_expr = q_expr + (2**q)*sp.symbols('x_'+str(n)+'_'+str(t)+'_'+str(q))
        
        w_n_t = (1/K)*q_expr
        w_t = np.append(w_t,w_n_t)
    
    return sp.Matrix([w_t])

N = 3
Nt = 2
Nq = 1
K = 2
Ntot = N*Nt*Nq

f_returns = np.random.rand(Nt,N)
cov = np.array([np.random.rand(N,N),np.random.rand(N,N)])
risk = 1/2
l = np.random.rand()

H = 0
for t in range(Nt):
    w_t = generateWt(t,N,Nq,K)
    mu_t = -1*sp.Matrix([f_returns[t]])
    cov_t = sp.Matrix(cov[t])
    
    H = H + mu_t.dot(w_t)
    H = H + risk*(w_t*cov_t).dot(w_t)

sp.init_printing()

H = sp.Poly(sp.expand(H))
qubo = generateQUBO(N,Nt,Nq,H)

print(H)
print('----------')
print(qubo)

#tx_expr = 0
#for t in range(Nt-1):
#    tx_expr = tx_expr + l*(generateWt(t+1,N,Nq,K) - generateWt(t,N,Nq,K))*(generateWt(t+1,N,Nq,K) - generateWt(t,N,Nq,K))

#H = H + tx_expr

#QUBO = np.zeros((Ntot,Ntot))

#asset_data = np.random.rand(365,N)
#forcasted_return = np.random.rand(Nt,N)
#covariances = np.array([np.random.rand(N,N),np.random.rand(N,N)])
#tx_costs = np.array([np.random.rand(N,N),np.random.rand(N,N)])

#print(forcasted_return)

['x_0_0_0' 'x_0_1_0' 'x_1_0_0' 'x_1_1_0' 'x_2_0_0' 'x_2_1_0' 'x_3_0_0'
 'x_3_1_0' 'x_4_0_0' 'x_4_1_0' 'x_5_0_0' 'x_5_1_0']
Poly(0.102370993134838*x_0_0_0**2 + 0.0631830950458628*x_0_0_0*x_1_0_0 + 0.141467513981981*x_0_0_0*x_2_0_0 + 0.201392249361744*x_0_0_0*x_3_0_0 + 0.134039011795002*x_0_0_0*x_4_0_0 + 0.130485095810829*x_0_0_0*x_5_0_0 - 0.0732286190157175*x_0_0_0 + 0.0634726328007583*x_0_1_0**2 + 0.184124568528616*x_0_1_0*x_1_1_0 + 0.148898497672739*x_0_1_0*x_2_1_0 + 0.175651720924981*x_0_1_0*x_3_1_0 + 0.0610248263105498*x_0_1_0*x_4_1_0 + 0.160517299193014*x_0_1_0*x_5_1_0 - 0.304095998949624*x_0_1_0 + 0.0971070215949068*x_1_0_0**2 + 0.094185585616219*x_1_0_0*x_2_0_0 + 0.0949395987575418*x_1_0_0*x_3_0_0 + 0.0318636274473638*x_1_0_0*x_4_0_0 + 0.147030508261686*x_1_0_0*x_5_0_0 - 0.484500697169221*x_1_0_0 + 0.0220005401114773*x_1_1_0**2 + 0.141531459536857*x_1_1_0*x_2_1_0 + 0.181960979557243*x_1_1_0*x_3_1_0 + 0.110522302032154*x_1_1_0*x_4_1_0 + 0.1782648234152*x_1_1_0*x_5_1_0 - 0.27990415

In [ ]:
# Convert the problem to a BQM
bqm = BinaryQuadraticModel.from_numpy_matrix(qubo)

# Define the sampler that will be used to run the problem
sampler = EmbeddingComposite(DWaveSampler())

# Run the problem on the sampler and print the results
sampleset = sampler.sample(bqm, num_reads = 10)
print(sampleset)

In [46]:
0.108192864114045-0.110824041458911

In [15]:
def Q1(forcasted_return):
    Q1_QUBO = np.zeros((Ntot,Ntot))
    np.fill_diagonal(Q1_QUBO,-1*forcasted_return)
        
    return Q1_QUBO

print(Q1(forcasted_return))

NameError: name 'forcasted_return' is not defined